# off-ball xT modeling

Based on Soccermatics off-ball xT ideas and DataBallPy Markov-chain xT structure.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from supabase import create_client
from mplsoccer import Pitch
from sklearn.linear_model import LogisticRegression

pd.set_option("display.max_columns", 100)

In [ ]:
GOAL_X = 105.0
GOAL_Y = 34.0
NX = 16
NY = 12

In [ ]:
# Edit MATCH_ID_INPUT (one match) and PLAYER_INPUT (optional)
# Edit SUPABASE_KEY if it changes
SUPABASE_URL = "https://cctasilkmirgtrvapggo.supabase.co"
SUPABASE_KEY = "sb_secret_k_EFKytwKeNCeBWtjyohsg_EcPyz3XP"

MATCH_ID_INPUT = "3812"
PLAYER_INPUT = ""
FETCH_BATCH_SIZE = 1000

client = create_client(SUPABASE_URL, SUPABASE_KEY)
print("Supabase client ready.")

In [ ]:
match_text = str(MATCH_ID_INPUT).strip()
if match_text == "":
    raise ValueError('Set MATCH_ID_INPUT to one match id, e.g. "3812".')

match_value = float(match_text)
if not match_value.is_integer():
    raise ValueError("MATCH_ID_INPUT must be an integer match id.")

match_id = int(match_value)

print(f"Fetching data for match {match_id}...")

passes_rows = []
offset = 0
while True:
    response = (
        client.table("passes")
        .select("*")
        .eq("match_id", match_id)
        .range(offset, offset + FETCH_BATCH_SIZE - 1)
        .execute()
    )
    batch = response.data or []
    passes_rows.extend(batch)
    if len(batch) < FETCH_BATCH_SIZE:
        break
    offset += FETCH_BATCH_SIZE

xt_rows = []
offset = 0
while True:
    response = (
        client.table("xt_events")
        .select("*")
        .eq("match_id", match_id)
        .range(offset, offset + FETCH_BATCH_SIZE - 1)
        .execute()
    )
    batch = response.data or []
    xt_rows.extend(batch)
    if len(batch) < FETCH_BATCH_SIZE:
        break
    offset += FETCH_BATCH_SIZE

frame_rows = []
offset = 0
while True:
    response = (
        client.table("tracking_frames")
        .select("*")
        .eq("game_id", match_id)
        .range(offset, offset + FETCH_BATCH_SIZE - 1)
        .execute()
    )
    batch = response.data or []
    frame_rows.extend(batch)
    if len(batch) < FETCH_BATCH_SIZE:
        break
    offset += FETCH_BATCH_SIZE

ball_rows = []
offset = 0
while True:
    response = (
        client.table("tracking_ball_positions")
        .select("*")
        .eq("game_id", match_id)
        .range(offset, offset + FETCH_BATCH_SIZE - 1)
        .execute()
    )
    batch = response.data or []
    ball_rows.extend(batch)
    if len(batch) < FETCH_BATCH_SIZE:
        break
    offset += FETCH_BATCH_SIZE

passes_df = pd.DataFrame(passes_rows)
xt_df = pd.DataFrame(xt_rows)
frames_df = pd.DataFrame(frame_rows)
ball_df = pd.DataFrame(ball_rows)

if passes_df.empty:
    raise ValueError(f"No passes found for match {match_id}.")
if xt_df.empty:
    raise ValueError(f"No xt_events found for match {match_id}.")
if frames_df.empty:
    raise ValueError(f"No tracking_frames found for match {match_id}.")
if ball_df.empty:
    raise ValueError(f"No tracking_ball_positions found for match {match_id}.")

for col in ["possession_event_id", "match_id"]:
    if col in passes_df.columns:
        passes_df[col] = pd.to_numeric(passes_df[col], errors="coerce")

for col in ["match_id", "possession_event_id", "game_event_id", "sequence", "team_id", "period"]:
    if col in xt_df.columns:
        xt_df[col] = pd.to_numeric(xt_df[col], errors="coerce")

for col in ["game_id", "frame_num", "possession_event_id", "game_event_id", "period"]:
    if col in frames_df.columns:
        frames_df[col] = pd.to_numeric(frames_df[col], errors="coerce")

for col in ["game_id", "frame_num", "x", "y", "z"]:
    if col in ball_df.columns:
        ball_df[col] = pd.to_numeric(ball_df[col], errors="coerce")

print(f"passes: {len(passes_df)}")
print(f"xt_events: {len(xt_df)}")
print(f"tracking_frames: {len(frames_df)}")
print(f"tracking_ball_positions: {len(ball_df)}")

In [ ]:
if "possession_event_id" not in xt_df.columns:
    raise ValueError("xt_events is missing possession_event_id.")
if "game_event_id" not in xt_df.columns:
    raise ValueError("xt_events is missing game_event_id.")
if "frame_num" not in frames_df.columns:
    raise ValueError("tracking_frames is missing frame_num.")
if "frame_num" not in ball_df.columns:
    raise ValueError("tracking_ball_positions is missing frame_num.")

xt_unique = (
    xt_df.sort_values(["sequence", "game_event_id"]).drop_duplicates("possession_event_id", keep="first").copy()
)

ball_map = ball_df.sort_values("frame_num").drop_duplicates("frame_num", keep="first")

frame_map_pos = (
    frames_df.dropna(subset=["possession_event_id"])
    .sort_values("frame_num")
    .drop_duplicates("possession_event_id", keep="first")
)
frame_map_pos = frame_map_pos.merge(
    ball_map[["frame_num", "x", "y", "z"]],
    on="frame_num",
    how="left",
)
frame_map_pos = frame_map_pos[["possession_event_id", "x", "y", "z"]]

frame_map_game = (
    frames_df.dropna(subset=["game_event_id"])
    .sort_values("frame_num")
    .drop_duplicates("game_event_id", keep="first")
)
frame_map_game = frame_map_game.merge(
    ball_map[["frame_num", "x", "y", "z"]],
    on="frame_num",
    how="left",
)
frame_map_game = frame_map_game[["game_event_id", "x", "y", "z"]]

xt_event_xy = xt_unique.merge(
    frame_map_pos.rename(columns={"x": "x_pos", "y": "y_pos", "z": "z_pos"}),
    on="possession_event_id",
    how="left",
)
xt_event_xy = xt_event_xy.merge(
    frame_map_game.rename(columns={"x": "x_game", "y": "y_game", "z": "z_game"}),
    on="game_event_id",
    how="left",
)

xt_event_xy["x_raw"] = xt_event_xy["x_pos"].fillna(xt_event_xy["x_game"])
xt_event_xy["y_raw"] = xt_event_xy["y_pos"].fillna(xt_event_xy["y_game"])
xt_event_xy["z_raw"] = xt_event_xy["z_pos"].fillna(xt_event_xy["z_game"])

coord_df = xt_event_xy[xt_event_xy["x_raw"].notna() & xt_event_xy["y_raw"].notna()].copy()
if coord_df.empty:
    raise ValueError("No possession events with tracking coordinates for this match.")

x_q01, x_q99 = np.nanpercentile(coord_df["x_raw"], [1, 99])
y_q01, y_q99 = np.nanpercentile(coord_df["y_raw"], [1, 99])

if -60 <= x_q01 and x_q99 <= 60 and -40 <= y_q01 and y_q99 <= 40:
    coord_mode = "centered_meters"
    xt_event_xy["x_pitch"] = xt_event_xy["x_raw"] + 52.5
    xt_event_xy["y_pitch"] = xt_event_xy["y_raw"] + 34.0
elif -5 <= x_q01 and x_q99 <= 110 and -5 <= y_q01 and y_q99 <= 73:
    coord_mode = "pitch_like_meters"
    xt_event_xy["x_pitch"] = xt_event_xy["x_raw"]
    xt_event_xy["y_pitch"] = xt_event_xy["y_raw"]
else:
    coord_mode = "scaled_from_raw"
    x_min, x_max = np.nanpercentile(coord_df["x_raw"], [1, 99])
    y_min, y_max = np.nanpercentile(coord_df["y_raw"], [1, 99])
    x_span = max(x_max - x_min, 1e-6)
    y_span = max(y_max - y_min, 1e-6)
    xt_event_xy["x_pitch"] = (xt_event_xy["x_raw"] - x_min) * (105.0 / x_span)
    xt_event_xy["y_pitch"] = (xt_event_xy["y_raw"] - y_min) * (68.0 / y_span)

xt_event_xy["x_pitch"] = xt_event_xy["x_pitch"].clip(0, 105)
xt_event_xy["y_pitch"] = xt_event_xy["y_pitch"].clip(0, 68)

if "team_id" in xt_event_xy.columns and "period" in xt_event_xy.columns:
    shot_dir = (
        xt_event_xy[xt_event_xy["possession_event_type"] == "SH"]
        .dropna(subset=["team_id", "period", "x_pitch"])
        .groupby(["team_id", "period"], as_index=False)
        .agg(median_shot_x=("x_pitch", "median"))
    )
    fallback_dir = (
        xt_event_xy.dropna(subset=["team_id", "period", "x_pitch"])
        .groupby(["team_id", "period"], as_index=False)
        .agg(median_event_x=("x_pitch", "median"))
    )
    dir_df = fallback_dir.merge(shot_dir, on=["team_id", "period"], how="left")
    dir_df["direction_value"] = dir_df["median_shot_x"].fillna(dir_df["median_event_x"])
    dir_df["attacking_right"] = dir_df["direction_value"] >= 52.5

    xt_event_xy = xt_event_xy.merge(
        dir_df[["team_id", "period", "attacking_right"]],
        on=["team_id", "period"],
        how="left",
    )
    xt_event_xy["attacking_right"] = xt_event_xy["attacking_right"].fillna(True)
else:
    xt_event_xy["attacking_right"] = True

xt_event_xy["x_std"] = np.where(
    xt_event_xy["attacking_right"],
    xt_event_xy["x_pitch"],
    105 - xt_event_xy["x_pitch"],
)
xt_event_xy["y_std"] = np.where(
    xt_event_xy["attacking_right"],
    xt_event_xy["y_pitch"],
    68 - xt_event_xy["y_pitch"],
)

xt_event_xy = xt_event_xy[xt_event_xy["x_std"].notna() & xt_event_xy["y_std"].notna()].copy()

print(f"Coordinate mode: {coord_mode}")
print(f"xt events with coordinates: {len(xt_event_xy)} / {len(xt_unique)}")
print(f"x_std range: {xt_event_xy['x_std'].min():.2f} to {xt_event_xy['x_std'].max():.2f}")
print(f"y_std range: {xt_event_xy['y_std'].min():.2f} to {xt_event_xy['y_std'].max():.2f}")

In [ ]:
xt_event_xy = xt_event_xy.sort_values(["sequence", "game_event_id"]).copy()
xt_event_xy["next_possession_event_id"] = xt_event_xy["possession_event_id"].shift(-1)

coord_map = xt_event_xy[["possession_event_id", "x_std", "y_std"]].drop_duplicates("possession_event_id")

pass_events = xt_event_xy[xt_event_xy["possession_event_type"] == "PA"].copy()
pass_events = pass_events.merge(
    passes_df[[
        "possession_event_id",
        "pass_outcome_type",
        "pass_type",
        "passer_player_name",
        "receiver_player_name",
        "target_player_name",
    ]],
    on="possession_event_id",
    how="left",
    suffixes=("", "_passes"),
)

for col in [
    "pass_outcome_type",
    "pass_type",
    "passer_player_name",
    "receiver_player_name",
    "target_player_name",
]:
    col_passes = f"{col}_passes"
    col_x = f"{col}_x"
    col_y = f"{col}_y"

    if col not in pass_events.columns:
        if col_passes in pass_events.columns:
            pass_events[col] = pass_events[col_passes]
        elif col_x in pass_events.columns:
            pass_events[col] = pass_events[col_x]
        elif col_y in pass_events.columns:
            pass_events[col] = pass_events[col_y]

    if col in pass_events.columns and col_passes in pass_events.columns:
        pass_events[col] = pass_events[col].fillna(pass_events[col_passes])

    if col in pass_events.columns and col_x in pass_events.columns:
        pass_events[col] = pass_events[col].fillna(pass_events[col_x])

    if col in pass_events.columns and col_y in pass_events.columns:
        pass_events[col] = pass_events[col].fillna(pass_events[col_y])

if "pass_outcome_type" not in pass_events.columns:
    pass_events["pass_outcome_type"] = np.nan
if "pass_type" not in pass_events.columns:
    pass_events["pass_type"] = np.nan
if "passer_player_name" not in pass_events.columns:
    pass_events["passer_player_name"] = "Unknown"
if "receiver_player_name" not in pass_events.columns:
    pass_events["receiver_player_name"] = np.nan
if "target_player_name" not in pass_events.columns:
    pass_events["target_player_name"] = np.nan

pass_events = pass_events.merge(
    coord_map.rename(columns={"x_std": "x_start", "y_std": "y_start"}),
    on="possession_event_id",
    how="left",
)
pass_events = pass_events.merge(
    coord_map.rename(
        columns={
            "possession_event_id": "next_possession_event_id",
            "x_std": "x_end",
            "y_std": "y_end",
        }
    ),
    on="next_possession_event_id",
    how="left",
)

pass_events = pass_events[pass_events["x_start"].notna() & pass_events["y_start"].notna()].copy()

shot_events = xt_event_xy[xt_event_xy["possession_event_type"] == "SH"].copy()

if pass_events.empty:
    raise ValueError("No pass events with start coordinates after joins.")
if shot_events.empty:
    raise ValueError("No shot events found in xt_events for this match.")

pass_events["x_start_bin"] = np.clip(np.floor(pass_events["x_start"] / 105 * NX).astype(int), 0, NX - 1)
pass_events["y_start_bin"] = np.clip(np.floor(pass_events["y_start"] / 68 * NY).astype(int), 0, NY - 1)

pass_events["has_end_coords"] = pass_events["x_end"].notna() & pass_events["y_end"].notna()
pass_events["x_end_bin"] = np.nan
pass_events["y_end_bin"] = np.nan

valid_end = pass_events["has_end_coords"]
pass_events.loc[valid_end, "x_end_bin"] = np.clip(
    np.floor(pass_events.loc[valid_end, "x_end"] / 105 * NX).astype(int),
    0,
    NX - 1,
)
pass_events.loc[valid_end, "y_end_bin"] = np.clip(
    np.floor(pass_events.loc[valid_end, "y_end"] / 68 * NY).astype(int),
    0,
    NY - 1,
)

shot_events["x_bin"] = np.clip(np.floor(shot_events["x_std"] / 105 * NX).astype(int), 0, NX - 1)
shot_events["y_bin"] = np.clip(np.floor(shot_events["y_std"] / 68 * NY).astype(int), 0, NY - 1)

actions_df = pd.concat(
    [
        pass_events[["x_start_bin", "y_start_bin"]].rename(columns={"x_start_bin": "x_bin", "y_start_bin": "y_bin"}),
        shot_events[["x_bin", "y_bin"]],
    ],
    ignore_index=True,
)

action_counts = (
    actions_df.groupby(["y_bin", "x_bin"]).size().reset_index(name="n_actions")
)
shot_counts = (
    shot_events.groupby(["y_bin", "x_bin"]).size().reset_index(name="n_shots")
)
goal_counts = (
    shot_events[shot_events["shot_outcome_type"] == "G"]
    .groupby(["y_bin", "x_bin"])
    .size()
    .reset_index(name="n_goals")
)
move_counts = (
    pass_events[pass_events["pass_outcome_type"] == "C"]
    .groupby(["y_start_bin", "x_start_bin"])
    .size()
    .reset_index(name="n_successful_moves")
    .rename(columns={"y_start_bin": "y_bin", "x_start_bin": "x_bin"})
)

cell_df = action_counts.merge(shot_counts, on=["y_bin", "x_bin"], how="left")
cell_df = cell_df.merge(goal_counts, on=["y_bin", "x_bin"], how="left")
cell_df = cell_df.merge(move_counts, on=["y_bin", "x_bin"], how="left")
cell_df = cell_df.fillna(0)

shot_prob = np.zeros((NY, NX))
move_prob = np.zeros((NY, NX))
goal_prob = np.zeros((NY, NX))

for row in cell_df.itertuples(index=False):
    yb = int(row.y_bin)
    xb = int(row.x_bin)
    n_actions = float(row.n_actions)
    n_shots = float(row.n_shots)
    n_goals = float(row.n_goals)
    n_moves = float(row.n_successful_moves)

    if n_actions > 0:
        shot_prob[yb, xb] = n_shots / n_actions
        move_prob[yb, xb] = n_moves / n_actions
    if n_shots > 0:
        goal_prob[yb, xb] = n_goals / n_shots

trans_df = pass_events[
    (pass_events["pass_outcome_type"] == "C")
    & (pass_events["has_end_coords"])
].copy()

trans_df["start_idx"] = (trans_df["y_start_bin"].astype(int) * NX + trans_df["x_start_bin"].astype(int))
trans_df["end_idx"] = (trans_df["y_end_bin"].astype(int) * NX + trans_df["x_end_bin"].astype(int))

transition_counts = trans_df.groupby(["start_idx", "end_idx"]).size().reset_index(name="n")
start_move_counts = transition_counts.groupby("start_idx")["n"].sum().to_dict()

n_cells = NY * NX
transition_matrix = np.zeros((n_cells, n_cells))
for row in transition_counts.itertuples(index=False):
    total = start_move_counts.get(int(row.start_idx), 0)
    if total > 0:
        transition_matrix[int(row.start_idx), int(row.end_idx)] = float(row.n) / float(total)

xT = np.zeros((NY, NX))
for _ in range(200):
    xT_flat = xT.reshape(-1)
    move_payoff = transition_matrix @ xT_flat
    move_payoff = move_payoff.reshape(NY, NX)
    xT_new = shot_prob * goal_prob + move_prob * move_payoff
    if np.max(np.abs(xT_new - xT)) < 1e-6:
        xT = xT_new
        break
    xT = xT_new

print(f"Pass events used: {len(pass_events)}")
print(f"Successful pass transitions used: {len(trans_df)}")
print(f"Shot events used: {len(shot_events)}")
print(f"xT range: {xT.min():.4f} to {xT.max():.4f}")

In [ ]:
pass_events = pass_events.copy()

valid_end = pass_events["has_end_coords"]
pass_events.loc[valid_end, "x_end_bin"] = np.clip(
    np.floor(pass_events.loc[valid_end, "x_end"] / 105 * NX).astype(int),
    0,
    NX - 1,
)
pass_events.loc[valid_end, "y_end_bin"] = np.clip(
    np.floor(pass_events.loc[valid_end, "y_end"] / 68 * NY).astype(int),
    0,
    NY - 1,
)

pass_events["distance"] = np.hypot(
    pass_events["x_end"] - pass_events["x_start"],
    pass_events["y_end"] - pass_events["y_start"],
)
pass_events["dx"] = pass_events["x_end"] - pass_events["x_start"]
pass_events["dy_abs"] = np.abs(pass_events["y_end"] - pass_events["y_start"])
pass_events["is_complete"] = (pass_events["pass_outcome_type"] == "C").astype(int)

pass_events["start_xT"] = xT[
    pass_events["y_start_bin"].astype(int),
    pass_events["x_start_bin"].astype(int),
]

pass_events["end_xT"] = np.nan
pass_events.loc[valid_end, "end_xT"] = xT[
    pass_events.loc[valid_end, "y_end_bin"].astype(int),
    pass_events.loc[valid_end, "x_end_bin"].astype(int),
]

train_pass = pass_events[valid_end].dropna(subset=["distance", "dx", "dy_abs"]).copy()

if train_pass.empty:
    raise ValueError("No passes with both start/end coordinates for off-ball modeling.")

if train_pass["is_complete"].nunique() >= 2 and len(train_pass) >= 30:
    X = train_pass[["distance", "dx", "dy_abs"]]
    y = train_pass["is_complete"]
    receive_model = LogisticRegression(max_iter=2000)
    receive_model.fit(X, y)
    pass_events["p_receive"] = receive_model.predict_proba(
        pass_events[["distance", "dx", "dy_abs"]].fillna(0)
    )[:, 1]
else:
    base_rate = train_pass["is_complete"].mean()
    pass_events["p_receive"] = base_rate

pass_events["xT_gain"] = pass_events["end_xT"] - pass_events["start_xT"]
pass_events["xT_gain"] = pass_events["xT_gain"].fillna(0)
pass_events["off_ball_xT"] = pass_events["p_receive"] * pass_events["xT_gain"]
pass_events["off_ball_xT"] = pass_events["off_ball_xT"].fillna(0)

pass_events["receiver_name"] = pass_events["receiver_player_name"].fillna(pass_events["target_player_name"])
pass_events["receiver_name"] = pass_events["receiver_name"].fillna("Unknown")
pass_events["passer_name"] = pass_events["passer_player_name"].fillna("Unknown")

if str(PLAYER_INPUT).strip() != "":
    player_text = str(PLAYER_INPUT).strip().lower()
    pass_view = pass_events[
        pass_events["receiver_name"].str.lower().str.contains(player_text, na=False)
        | pass_events["passer_name"].str.lower().str.contains(player_text, na=False)
    ].copy()
else:
    pass_view = pass_events.copy()

pass_view = pass_view.sort_values("off_ball_xT", ascending=False)

receiver_leaderboard = (
    pass_events.groupby("receiver_name", as_index=False)
    .agg(
        off_ball_xT_total=("off_ball_xT", "sum"),
        mean_off_ball_xT=("off_ball_xT", "mean"),
        n_pass_targets=("off_ball_xT", "size"),
    )
    .sort_values("off_ball_xT_total", ascending=False)
)

print(f"Passes with end coordinates: {int(valid_end.sum())} / {len(pass_events)}")
print(f"Average receive probability: {pass_events['p_receive'].mean():.3f}")
print(f"Average xT gain: {pass_events['xT_gain'].mean():.4f}")
print(f"Average off-ball xT: {pass_events['off_ball_xT'].mean():.4f}")

cols = [
    "passer_name",
    "receiver_name",
    "pass_outcome_type",
    "distance",
    "start_xT",
    "end_xT",
    "xT_gain",
    "p_receive",
    "off_ball_xT",
]
pass_view.head(20)[cols]

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

pitch = Pitch(
    pitch_type="custom",
    pitch_length=105,
    pitch_width=68,
    pitch_color="#22312b",
    line_color="white",
    linewidth=2,
)
pitch.draw(ax=ax1)

hm = ax1.imshow(
    xT,
    extent=(0, 105, 0, 68),
    origin="lower",
    cmap="Reds",
    alpha=0.78,
    aspect="auto",
)

plot_passes = pass_view[
    pass_view["has_end_coords"]
    & pass_view["x_start"].notna()
    & pass_view["y_start"].notna()
    & pass_view["x_end"].notna()
    & pass_view["y_end"].notna()
].copy()
plot_passes = plot_passes.sort_values("off_ball_xT", ascending=False).head(40)

if not plot_passes.empty:
    off_vals = plot_passes["off_ball_xT"].values
    if np.nanmax(np.abs(off_vals)) > 0:
        norm = (off_vals - np.nanmin(off_vals)) / (np.nanmax(off_vals) - np.nanmin(off_vals) + 1e-9)
    else:
        norm = np.zeros(len(off_vals))

    for i, row in enumerate(plot_passes.itertuples(index=False)):
        alpha_val = 0.35 + 0.55 * norm[i]
        color_val = "#7CFC00" if row.off_ball_xT >= 0 else "#6fa8dc"
        ax1.annotate(
            "",
            xy=(row.x_end, row.y_end),
            xytext=(row.x_start, row.y_start),
            arrowprops=dict(arrowstyle="->", color=color_val, lw=1.8, alpha=alpha_val),
        )

cbar = plt.colorbar(hm, ax=ax1, fraction=0.046, pad=0.04)
cbar.set_label("xT", rotation=270, labelpad=18)

ax1.set_title(
    f"Match {match_id}: xT grid + top off-ball passes",
    fontsize=14,
    fontweight="bold",
)

receiver_plot = receiver_leaderboard.head(12).copy()
ax2.barh(
    receiver_plot["receiver_name"][::-1],
    receiver_plot["off_ball_xT_total"][::-1],
    color="#ff6b6b",
    edgecolor="black",
)
ax2.set_xlabel("Total off-ball xT", fontweight="bold")
ax2.set_title("Top receivers by off-ball xT", fontweight="bold")
ax2.grid(axis="x", alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "=" * 60)
print("OFF-BALL XT SUMMARY")
print("=" * 60)
print(f"Match ID: {match_id}")
print(f"Pass events modeled: {len(pass_events)}")
print(f"Passes with end coordinates: {int(pass_events['has_end_coords'].sum())}")
print(f"Mean receive probability: {pass_events['p_receive'].mean():.3f}")
print(f"Mean xT gain: {pass_events['xT_gain'].mean():.4f}")
print(f"Mean off-ball xT: {pass_events['off_ball_xT'].mean():.4f}")
print(f"Total off-ball xT: {pass_events['off_ball_xT'].sum():.3f}")

if str(PLAYER_INPUT).strip() != "":
    print(f"Filtered player search: {PLAYER_INPUT}")

print("\nTop 10 receiver leaderboard:")
print(receiver_leaderboard.head(10).to_string(index=False))